## 读取图像，二值化
#### 1、方向检测、旋转？
#### 2、基本信息点查找 得到位置信息
#### 3、图像分割

In [1]:
import aircv as ac
import skimage.morphology as sm  # 膨胀与腐蚀
import matplotlib.pyplot as plt
from skimage import io,color
from skimage import data,filters  # filters  阈值处理
import numpy as np
import cv2  #  打开图像不会 旋转img=cv2.imread(file, 1)
# file = "/Users/zhoudaoxian/Documents/frame/沈13810387085.jpg"
# img = io.imread(file, True)
# img=cv2.imread(file, 1)


In [2]:
# img90=np.rot90(img)
def show_fig(title, sub, size,img):
    plt.figure(title, figsize=size)
    plt.subplot(sub)
    plt.imshow(img, plt.cm.gray)
    plt.title(title) # 图像题目
    plt.show()
# show_fig("ori",(111), (4,4), img)
# show_fig("ori", (20,20), img)

In [3]:
import pandas as pd
import StringIO
from PIL import Image
import pytesseract

In [5]:
im = Image.open("/Users/zhoudaoxian/Downloads/lp_data/落地页模版筛选图片和配色方法/401_早教/11.jpeg")
rel=pytesseract.image_to_boxes((im), lang='chi_sim')
rel = pd.read_csv(StringIO.StringIO(rel), sep=" ",  header=None)
# text = pytesseract.image_to_string((im), lang='chi_sim')
print(rel[1:10])

   0    1   2    3    4  5
1  合  203  84  244  126  0
2  作  261  84  293  125  0
3  新  299  84  340  125  0
4  模  346  84  388  125  0
5  式  413  84  435  125  0
6  提  172  17  469   49  0


In [6]:
boxs=pytesseract.image_to_boxes((im), lang='chi_sim')

In [13]:
print(boxs[1:100])
boxs[1:150]

 8 100 27 125 0
光 32 101 76 126 0
单 842 0 842 0 0
0 106 94 153 133 0
外 158 94 181 139 0
年 204 103 2


u' 8 100 27 125 0\n\u5149 32 101 76 126 0\n\u5355 842 0 842 0 0\n0 106 94 153 133 0\n\u5916 158 94 181 139 0\n\u5e74 204 103 228 128 0\n\u9a6c 245 102 266 142 0\n\u6708 283 104 303 127 0\n2'

In [71]:
box = pd.read_csv(StringIO.StringIO(boxs), sep=" ",  header=None)
box[1:10]

,0,1,2,3,4,5
1,光,32,101,76,126,0
2,单,842,0,842,0,0
3,0,106,94,153,133,0
4,外,158,94,181,139,0
5,年,204,103,228,128,0
6,马,245,102,266,142,0
7,月,283,104,303,127,0
8,2,312,105,327,131,0
9,7,327,81,354,146,0


## 腾讯AI

In [6]:
# caojiao
appid = "2111930335"
appkey = "ye1i7bjYBrCcB4Jb"

#  mengyao
appid = "2111931001"
appkey = "MSHOgpyd5ycBC5Nz"

# zdx
appid = '2111435318'
appkey = 'izx4fOqgtab2BGq1'

#  通用OCR能力：
# https://api.ai.qq.com/fcgi-bin/ocr/ocr_generalocr
# 手写OCR 
url = 'https://api.ai.qq.com/fcgi-bin/ocr/ocr_handwritingocr'

### # getReqSign ：根据 接口请求参数 和 应用密钥 计算 请求签名

In [7]:
import sys
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

import hashlib  
import time  
import random  
import string
import base64  
import requests
import cv2
import numpy as np
from urllib import urlencode
import json 

In [11]:
url = 'https://api.ai.qq.com/fcgi-bin/ocr/ocr_handwritingocr'  
# url = 'https://api.ai.qq.com/fcgi-bin/ocr/ocr_generalocr'  

path   = '/Users/zhoudaoxian/Desktop/test.jpg'

In [9]:
def get_params(img):
    time_stamp=str(int(time.time())) 
    nonce_str = ''.join(str(random.choice(range(10))) for _ in range(10))
    params = {'app_id':appid,                #请求包，需要根据不同的任务修改，基本相同
              'image':img,                    #文字类的任务可能是‘text’，由主函数传递进来
              'time_stamp':time_stamp,        #时间戳，都一样
              'nonce_str':nonce_str,          #随机字符串，都一样
             }
    sort_dict= sorted(params.items(), key=lambda item:item[0], reverse = False)  #字典排序
    sort_dict.append(('app_key',appkey))   #尾部添加appkey
    rawtext= urlencode(sort_dict).encode()  #urlencod编码
    sha = hashlib.md5()    
    sha.update(rawtext)
    md5text= sha.hexdigest().upper()        #MD5加密计算
    params['sign']=md5text                  #将签名赋值到sign
    return  params                          #返回请求包

In [10]:
#用opencv读入图片
def txocr(path):
    frame=cv2.imread(path)
    nparry_encode = cv2.imencode('.jpg', frame)[1]
    data_encode = np.array(nparry_encode)
    img = base64.b64encode(data_encode)    #得到API可以识别的字符串
    params = get_params(img)    #获取鉴权签名并获取请求参数
    res = requests.post(url,params).json()
    if res:
        return res
    return None

In [14]:
params = get_params(img)
print(params)
res = txocr(path)

{'time_stamp': '1551950919', 'image': None, 'nonce_str': '1451402990', 'app_id': '2111435318', 'sign': 'B0175AAD2074A7D449FC947A7D82DAE0'}


In [13]:
print(len(res['data'][u'item_list']))
a = res['data'][u'item_list']
for _ in a:
    print(_['itemstring'])
    break
print(a[0].keys())
for _ in a[0]:
    print(_)
    print(a[0][_])

1
回腾讯图像识别+本地ocr.ipyn X
[u'item', u'itemstring', u'itemcoord', u'words']
item

itemstring
回腾讯图像识别+本地ocr.ipyn X
itemcoord
[{u'y': 20, u'x': 21, u'height': 30, u'width': 388}]
words
[{u'confidence': 0.403824, u'character': u'\u56de'}, {u'confidence': 0.999777, u'character': u'\u817e'}, {u'confidence': 0.999945, u'character': u'\u8baf'}, {u'confidence': 0.999725, u'character': u'\u56fe'}, {u'confidence': 0.99874, u'character': u'\u50cf'}, {u'confidence': 0.999931, u'character': u'\u8bc6'}, {u'confidence': 0.999527, u'character': u'\u522b'}, {u'confidence': 0.999871, u'character': u'+'}, {u'confidence': 0.999917, u'character': u'\u672c'}, {u'confidence': 0.999992, u'character': u'\u5730'}, {u'confidence': 0.94651, u'character': u'o'}, {u'confidence': 0.998367, u'character': u'c'}, {u'confidence': 0.999868, u'character': u'r'}, {u'confidence': 0.953986, u'character': u'.'}, {u'confidence': 0.99505, u'character': u'i'}, {u'confidence': 0.999097, u'character': u'p'}, {u'confidence': 0.999015, 

## 处理NOdeJS输出文件

In [11]:
names = "/Users/zhoudaoxian/Desktop/frame/data/names.js"
tells = "/Users/zhoudaoxian/Desktop/frame/data/tells.js"

#   taobao 浏览器模拟爬虫

In [17]:
import requests    #  发送请求必备
url = "https://www.baidu.com"
# http请求
response = requests.get(url)
# html源码
html = response.text
print(html[0:100])

<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charse


## 分析页面，找出可以格式化的json 信息

In [ ]:
import re
content = re.findall(r'g_page_config = (.*?) g_srp_loadCss',html,re.S)[0].strip()[:-1]
#格式化
content = json.loads(content)

消息的加载方式：同步和异步 前面的36条数据是随网页一同加载，后面12条是通过异步进行加载的

cookie保持::

In [18]:
#cookie保持
cookies = response.cookies
#首页12条异步
url2 = 'https://s.taobao.com/api?_ksTS=1524836494360_224&callback=jsonp225&ajax=true&m=customized&sourceId=tb.index&q=python&spm=a21bo.2017.201856-taobao-item.1&s=36&imgfile=&initiative_id=tbindexz_20170306&bcoffset=-1&commend=all&ie=utf8&rn=efedc6cda629c8a38008aff6f017b934&ssid=s5-e&search_type=item'
 
response2 = requests.get(url2,cookies=cookies)
html2=response2.text
 
content = re.findall(r'{.*}',html2)[0]

NameError: name 're' is not defined